In [11]:
import sys
import nltk
import sklearn
import pandas
import numpy

import pandas as pd
import numpy as np
df = pd.read_csv('Bug Reports Dataset.csv')
print (df.head(5))

   Bug ID Product Component          Assignee    Status Classification  \
0  528905     JDT      Core    jdt-core-inbox  RESOLVED        Eclipse   
1  530231     JDT      Core    jdt-core-inbox  RESOLVED        Eclipse   
2  530758     JDT      Core          jarthana  VERIFIED        Eclipse   
3  531990     JDT      Core    jdt-core-inbox  VERIFIED        Eclipse   
4  532137     JDT      Core  register.eclipse  VERIFIED        Eclipse   

  Priority            Opened  \
0       P1  12/18/2017 11:46   
1       P1    1/24/2018 5:16   
2       P1    2/5/2018 20:38   
3       P1     3/5/2018 0:49   
4       P1    3/7/2018 16:17   

                                             Summary       Category  
0  JDT UI Gerrit failing with "invalid location f...  Configuration  
1  Compilation failure in M20180123-0400 in jdt.c...  Configuration  
2                    Build failure in I20180205-2000  Configuration  
3                    Build failure in I20180304-2000  Configuration  
4  Todays up

In [12]:
classes = df['Category']
print(classes.value_counts())

Program_Anomaly      498
GUI                  465
Test_Code            336
Performance          316
Configuration        281
NetworkOrSecurity    242
Name: Category, dtype: int64


In [13]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
Y = encoder.fit_transform (classes)
print(classes[:8])
print(Y[:8])
priority = df['Category']
print(priority[:10])

0    Configuration
1    Configuration
2    Configuration
3    Configuration
4    Configuration
5    Configuration
6    Configuration
7    Configuration
Name: Category, dtype: object
[0 0 0 0 0 0 0 0]
0    Configuration
1    Configuration
2    Configuration
3    Configuration
4    Configuration
5    Configuration
6    Configuration
7    Configuration
8    Configuration
9    Configuration
Name: Category, dtype: object


In [14]:
# Categorical boolean mask
categorical_feature_mask = df.dtypes == object

In [15]:
categorical_feature_mask

Bug ID            False
Product            True
Component          True
Assignee           True
Status             True
Classification     True
Priority           True
Opened             True
Summary            True
Category           True
dtype: bool

In [20]:
# filter categorical columns using mask and turn it into a list
categorical_cols = df.columns[categorical_feature_mask].tolist()
categorical_cols

['Product',
 'Component',
 'Assignee',
 'Status',
 'Classification',
 'Priority',
 'Opened',
 'Summary',
 'Category']

In [21]:
pri_col = []
for i in categorical_cols:
    if i != 'Category':
        pri_col.append(i)
print(pri_col)

['Product', 'Component', 'Assignee', 'Status', 'Classification', 'Priority', 'Opened', 'Summary']


In [22]:
le = LabelEncoder()
df[pri_col] = df[pri_col].apply(lambda col: le.fit_transform(col))
df[pri_col].head(10)

,Product,Component,Assignee,Status,Classification,Priority,Opened,Summary
0,44,41,144,4,4,0,475,691
1,44,41,144,4,4,0,82,199
2,44,41,141,6,4,0,717,133
3,44,41,144,6,4,0,931,134
4,44,41,273,6,4,0,948,1179
5,44,41,141,6,4,0,1367,132
6,22,41,7,1,8,0,105,159
7,14,272,227,2,9,0,1407,264
8,14,98,227,5,9,0,149,1871
9,14,116,227,2,9,0,1610,343


In [24]:
df.Category.value_counts()

4    498
1    465
5    336
3    316
0    281
2    242
Name: Category, dtype: int64

In [10]:
from sklearn.utils import resample
df_minority0 = df[df.Priority==0]
df_minority1 = df[df.Priority==1]
df_minority2 = df[df.Priority==2]
df_minority3 = df[df.Priority==3]
df_minority4 = df[df.Priority==4]

In [505]:
df_minority_upsampled0 = resample(df_minority0, 
                                 replace=True,     # sample with replacement
                                 n_samples=400)    # to match majority class
df_minority_upsampled1 = resample(df_minority1, 
                                 replace=True,     # sample with replacement
                                 n_samples=200)    # to match majority class
df_minority_upsampled2 = resample(df_minority2, 
                                 replace=True,     # sample with replacement
                                 n_samples=100)    # to match majority class
df_minority_upsampled3 = resample(df_minority3, 
                                 replace=True,     # sample with replacement
                                 n_samples=300)    # to match majority class
df_minority_upsampled4 = resample(df_minority4, 
                                 replace=True,     # sample with replacement
                                 n_samples=200)

In [506]:
df = pd.concat([df, df_minority_upsampled0, df_minority_upsampled1, df_minority_upsampled2, df_minority_upsampled3, df_minority_upsampled4])

In [507]:
df.Priority.value_counts()

2    1025
1     739
0     694
3     455
4     425
Name: Priority, dtype: int64

In [69]:
X  = df[['Product','Classification','Component', 'Priority', 'Assignee','Status']]

In [70]:
print(X.shape)

(2138, 6)


In [71]:
Y = df['Category']

In [72]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [73]:
print(Y.value_counts())

4    498
1    465
5    336
3    316
0    281
2    242
Name: Category, dtype: int64


In [74]:
from sklearn.ensemble import RandomForestClassifier
text_classifier = RandomForestClassifier(n_estimators=100, random_state=0)  
text_classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [75]:
predictions = text_classifier.predict(X_test)

In [76]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,predictions))  
print(classification_report(y_test,predictions))  
print(accuracy_score(y_test, predictions))

[[19  2  4 11 12  6]
 [ 6 53  2 12  5  6]
 [ 6  1 29  7  6  4]
 [ 5  9  5 42 11  4]
 [ 5  8  3  4 71  8]
 [ 5 10  5  4 19 19]]
              precision    recall  f1-score   support

           0       0.41      0.35      0.38        54
           1       0.64      0.63      0.63        84
           2       0.60      0.55      0.57        53
           3       0.53      0.55      0.54        76
           4       0.57      0.72      0.64        99
           5       0.40      0.31      0.35        62

    accuracy                           0.54       428
   macro avg       0.53      0.52      0.52       428
weighted avg       0.54      0.54      0.54       428

0.544392523364486


In [77]:
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)

In [78]:
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred)) 

[[ 8 13  7  0 26  0]
 [ 2 41 15  3 22  1]
 [ 5  4 24  5 13  2]
 [ 6 16 13 16 24  1]
 [ 6 21  5  0 61  6]
 [ 1 17  4  0 30 10]]
              precision    recall  f1-score   support

           0       0.29      0.15      0.20        54
           1       0.37      0.49      0.42        84
           2       0.35      0.45      0.40        53
           3       0.67      0.21      0.32        76
           4       0.35      0.62      0.44        99
           5       0.50      0.16      0.24        62

    accuracy                           0.37       428
   macro avg       0.42      0.35      0.34       428
weighted avg       0.42      0.37      0.35       428

0.37383177570093457


In [79]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logreg = LogisticRegression(solver='liblinear', random_state=0)
#fit the model
logreg.fit(X_train, y_train)
y_pred_test = logreg.predict(X_test)

In [80]:
print(confusion_matrix(y_test,y_pred_test))  
print(classification_report(y_test,y_pred_test))  
print(accuracy_score(y_test, y_pred_test)) 

[[ 5 12  6  3 28  0]
 [ 1 49  7  6 20  1]
 [ 3  8 15  7 17  3]
 [ 4 21  5 14 31  1]
 [ 1 27  1  6 63  1]
 [ 1 18  2  3 35  3]]
              precision    recall  f1-score   support

           0       0.33      0.09      0.14        54
           1       0.36      0.58      0.45        84
           2       0.42      0.28      0.34        53
           3       0.36      0.18      0.24        76
           4       0.32      0.64      0.43        99
           5       0.33      0.05      0.08        62

    accuracy                           0.35       428
   macro avg       0.36      0.30      0.28       428
weighted avg       0.35      0.35      0.30       428

0.34813084112149534


In [81]:
from sklearn.tree import DecisionTreeClassifier 
dtree_model = DecisionTreeClassifier(criterion = 'entropy').fit(X_train, y_train) 
dtree_predictions = dtree_model.predict(X_test) 
print("Accuracy:",metrics.accuracy_score(y_test, dtree_predictions))

Accuracy: 0.5373831775700935


In [82]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,dtree_predictions))  
print(classification_report(y_test,dtree_predictions))  
print(accuracy_score(y_test, dtree_predictions))

[[25  2  7 10  6  4]
 [ 9 52  3 12  4  4]
 [ 5  3 31  4  7  3]
 [ 9 12  4 42  5  4]
 [11 12  3  7 60  6]
 [ 7  9  5  6 15 20]]
              precision    recall  f1-score   support

           0       0.38      0.46      0.42        54
           1       0.58      0.62      0.60        84
           2       0.58      0.58      0.58        53
           3       0.52      0.55      0.54        76
           4       0.62      0.61      0.61        99
           5       0.49      0.32      0.39        62

    accuracy                           0.54       428
   macro avg       0.53      0.52      0.52       428
weighted avg       0.54      0.54      0.54       428

0.5373831775700935
